In [1]:
import pickle
import json
from tqdm import tqdm
import numpy as np

In [2]:
global diffs

with open('diffs.pkl', 'rb') as f:
    diffs = pickle.load(f)

In [3]:
def reject_outliers(arr, m=31/18):
    if len(arr) == 1:
        return arr
    q1, q3 = np.nanpercentile(arr, [25, 75])
    iqr = q3 - q1
    lb = q1 - m * iqr
    ub = q3 + m * iqr
    if len(set(arr)) > 1:
        arr = arr[(arr > lb) & (arr < ub)]
    assert len(arr) > 0
    return arr

In [4]:
avg_diffs = {}

for char, follows in tqdm(diffs.items()):
    avg_diffs[char] = {}
    for letter, vals in follows.items():
        if not np.isnan(vals).all():
            try:
                data = reject_outliers(np.array(vals))
                avg_diffs[char][letter] = {
                    'mean': np.nanmean(data),
                    'std': np.nanstd(data)
                }
            except:
                pass

100%|██████████████████████████████████████████| 138/138 [00:34<00:00,  3.97it/s]


In [5]:
with open('avg_diffs.json', 'w') as f:
    json.dump(avg_diffs, f, ensure_ascii=False, indent=4)